# Data Collection
- 2001년 11월 31일 ~ 2022년 11월 30일 (총 253개월 데이터)
- KOSPI 전종목에 해당, 기간 내 상장되고 폐지되는 종목도 모두 포함

In [1]:
import pandas as pd
import datetime
import time
from dateutil.relativedelta import relativedelta
from pykrx import stock
import FinanceDataReader as fdr
import numpy as np

## Important Methods

In [3]:
# 장 개시일 확인하는 메소드
# 22년 1월 29일은 휴장일이지만 가까운 장 개시일을 찾아줌 : 20220128
stock.get_nearest_business_day_in_a_week("20220129") 

'20220128'

In [4]:
# 종가, 시가총액, 상장주식수, 월별 누적 거래량(매월말) 불러오기
stock.get_market_cap("20220128")

,종가,시가총액,거래량,거래대금,상장주식수
티커,,,,,
005930,73300,437585060915000,21367447,1552585967650,5969782550
373220,450000,105300000000000,4559773,2104994467000,234000000
000660,120500,87724284982500,4776510,568486717500,728002365
005935,66300,54557388210000,2335000,155545527020,822886700
035420,310000,50855216350000,974351,297142295500,164049085
...,...,...,...,...,...
267810,1340,2394580000,116,155940,1787000
215050,1100,2333832600,1103,1149100,2121666
329020,1995,1666128240,21,40945,835152


In [5]:
# 티커와 기간을 입력하면 해당 기간 해당 티커의 정보를 불러옴(조정종가 반영) 
stock.get_market_ohlcv("20011130", "20221031", '005930', adjusted=True)

,시가,고가,저가,종가,거래량,거래대금,등락률
날짜,,,,,,,
2001-11-30,4460,4460,4330,4360,624634,136923277000,3.81
2001-12-03,4430,4500,4290,4490,657275,143783687500,2.98
2001-12-04,4530,4610,4440,4600,654850,149396130000,2.45
2001-12-05,4760,5290,4740,5290,2302638,585311461000,15.00
2001-12-06,5560,5860,5350,5500,2292641,645975560000,3.97
...,...,...,...,...,...,...,...
2022-10-25,57000,58600,57000,57700,17405678,1009512119302,0.35
2022-10-26,58000,59600,57700,59400,20824967,1227076491560,2.95
2022-10-27,59700,60100,58900,59500,21756712,1296030977930,0.17


## Data Scraping Process
<br>2001년 11월 31일 ~ 2022년 11월 30일 (총 253개월 데이터)
<br>KOSPI 전종목에 해당, 기간 내 상장되고 폐지되는 종목도 모두 포함

1. 월말 시총 : df_cap
2. 월별 Turn Over : df_TO = df_q / df_num
   - 2.1 월별 총 누적 거래량 : df_q
   - 2.2  월말 총 주식 발행수 : df_num
3. 일별 조정종가 : adj_close
4. 월말 조정종가 : df_p 

In [6]:
#매월 마지막 날 개장일의 날짜를 담는 list : days
days = list()
start = datetime.datetime(2001,12,1)
for i in range(253): #월말의 날짜를 먼저담고
    days.append(start - relativedelta(days = 1))
    start = start + relativedelta(months = 1) 
    
for i in range(len(days)): #월말 날짜와 가장 가까운 이전 개장일 정보를 담음
    temp = days[i].strftime("%Y%m%d")
    print(temp, end = " ")
    time.sleep(1)
    days[i] = stock.get_nearest_business_day_in_a_week(temp) #최종적으로 매월 마지막 개장일의 날짜를 담은 list

20011130 20011231 20020131 20020228 20020331 20020430 20020531 20020630 20020731 20020831 20020930 20021031 20021130 20021231 20030131 20030228 20030331 20030430 20030531 20030630 20030731 20030831 20030930 20031031 20031130 20031231 20040131 20040229 20040331 20040430 20040531 20040630 20040731 20040831 20040930 20041031 20041130 20041231 20050131 20050228 20050331 20050430 20050531 20050630 20050731 20050831 20050930 20051031 20051130 20051231 20060131 20060228 20060331 20060430 20060531 20060630 20060731 20060831 20060930 20061031 20061130 20061231 20070131 20070228 20070331 20070430 20070531 20070630 20070731 20070831 20070930 20071031 20071130 20071231 20080131 20080229 20080331 20080430 20080531 20080630 20080731 20080831 20080930 20081031 20081130 20081231 20090131 20090228 20090331 20090430 20090531 20090630 20090731 20090831 20090930 20091031 20091130 20091231 20100131 20100228 20100331 20100430 20100531 20100630 20100731 20100831 20100930 20101031 20101130 20101231 20110131 2

In [7]:
df_cap = pd.DataFrame() #월말 시총
df_num = pd.DataFrame() #월말 주식 발행량
df_q = pd.DataFrame() #월 누적 총 거래량

for i in days[]:
    time.sleep(1)
    print(i, end=" ")

    df = stock.get_market_cap(i)[['시가총액']] ; df.rename(columns = {'시가총액':i},inplace = True)
    df_cap = pd.concat([df_cap , df], axis = 1)
    
    df = stock.get_market_cap(i)[['상장주식수']] ; df.rename(columns = {'상장주식수':i},inplace = True)
    df_num = pd.concat([df_num , df], axis = 1)
    
    month_day = list()
    aa = i
    aa = datetime.datetime.strptime(aa, "%Y%m%d") ; stop = aa.month 
    while(True):
        aa = aa.strftime("%Y%m%d")
        aa = stock.get_nearest_business_day_in_a_week(aa)
        aa = datetime.datetime.strptime(aa, "%Y%m%d")
        if stop != aa.month : break
        aa = aa.strftime("%Y%m%d")
        month_day.append(aa) ; 
        aa = (datetime.datetime.strptime(aa, "%Y%m%d") - relativedelta(days = 1))

    temp_df = pd.DataFrame()
    for j in month_day:
        df = stock.get_market_cap(j)[['거래량']] ; df.rename(columns = {'거래량':j},inplace = True)
        temp_df = pd.concat([temp_df , df], axis = 1) ; temp_df = temp_df.sum(axis = 1)
        temp_df = pd.DataFrame(temp_df)
        temp_df.rename(columns = {0:i}, inplace = True)
    df_q = pd.concat([df_q,temp_df], axis = 1)
print("-----------------------------------------------월 총 누적 거래량-----------------------------------------------")
display(df_q)
print("-----------------------------------------------월말 시총-----------------------------------------------")
display(df_cap)
print("-----------------------------------------------월말 주식 발행 수-----------------------------------------------")
display(df_num)

-----------------------------------------------월 총 누적 거래량-----------------------------------------------


,20011130,20011228,20020131,20020228,20020329,20020430,20020531,20020628,20020731,20020830,...,20220228,20220331,20220429,20220531,20220630,20220729,20220831,20220930,20221031,20221130
티커,,,,,,,,,,,,,,,,,,,,,
005930,17058460.0,19202499.0,20370088.0,14298721.0,19244862.0,24635376.0,21390695.0,13929213.0,16814033.0,15035310.0,...,233201135.0,307432780.0,309880731.0,310129252.0,428473209.0,301826933.0,251681339.0,288615372.0,326537635.0,290682036.0
017670,6935909.0,5699755.0,7127119.0,3572625.0,5182411.0,5682311.0,5446338.0,3140815.0,6916929.0,4600389.0,...,11544932.0,12604532.0,16312219.0,15097497.0,16522976.0,11831889.0,17408500.0,17128150.0,9723797.0,10657162.0
030200,17520282.0,21838983.0,56612504.0,19829192.0,29485718.0,19895260.0,51153474.0,11451608.0,16445448.0,22714330.0,...,12867593.0,22941246.0,25399334.0,22802223.0,18317133.0,15178086.0,16571719.0,16910298.0,13457746.0,16003820.0
015760,54889900.0,42548450.0,46431310.0,37264005.0,89600307.0,60325753.0,50386739.0,29628861.0,36611863.0,24485496.0,...,33934899.0,56123467.0,32603911.0,39060677.0,37399691.0,21787061.0,24952987.0,26989496.0,25667043.0,37091835.0
060000,56549590.0,32678220.0,44950181.0,28435999.0,38535613.0,42559673.0,30603027.0,26998677.0,26808164.0,18971515.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3634.0
440200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2496032.0
438220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,823094.0


-----------------------------------------------월말 시총-----------------------------------------------


,20011130,20011228,20020131,20020228,20020329,20020430,20020531,20020628,20020731,20020830,...,20220228,20220331,20220429,20220531,20220630,20220729,20220831,20220930,20221031,20221130
티커,,,,,,,,,,,,,,,,,,,,,
005930,3.298958e+13,4.222061e+13,4.547419e+13,5.192672e+13,5.589079e+13,5.803774e+13,5.202116e+13,5.006687e+13,5.077855e+13,5.054948e+13,...,4.304213e+14,4.154969e+14,4.023633e+14,4.023633e+14,3.402776e+14,3.665446e+14,3.563960e+14,3.169955e+14,3.546051e+14,3.713205e+14
017670,2.353630e+13,2.389292e+13,2.282308e+13,2.317969e+13,2.585427e+13,2.242190e+13,2.411580e+13,2.402664e+13,2.059427e+13,2.108461e+13,...,1.194829e+13,1.245161e+13,1.247349e+13,1.249537e+13,1.137932e+13,1.172946e+13,1.140121e+13,1.111672e+13,1.096354e+13,1.094166e+13
030200,1.657780e+13,1.559437e+13,1.465777e+13,1.820124e+13,1.945004e+13,1.817002e+13,1.692122e+13,1.507924e+13,1.368996e+13,1.707732e+13,...,8.355578e+12,9.321692e+12,9.347803e+12,9.896138e+12,9.517525e+12,9.856971e+12,9.713359e+12,9.452247e+12,9.556692e+12,9.765582e+12
015760,1.472232e+13,1.389019e+13,1.395420e+13,1.405021e+13,1.584250e+13,1.568247e+13,1.613054e+13,1.408222e+13,1.369816e+13,1.347412e+13,...,1.489357e+13,1.454049e+13,1.482937e+13,1.498986e+13,1.447629e+13,1.434790e+13,1.344915e+13,1.290348e+13,1.075290e+13,1.328866e+13
060000,1.468518e+13,1.492493e+13,1.729254e+13,1.735248e+13,1.663321e+13,1.871120e+13,1.991837e+13,1.855236e+13,1.747226e+13,1.785347e+13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.865875e+10
440200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.209688e+10
438220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.128000e+10


-----------------------------------------------월말 주식 발행 수-----------------------------------------------


,20011130,20011228,20020131,20020228,20020329,20020430,20020531,20020628,20020731,20020830,...,20220228,20220331,20220429,20220531,20220630,20220729,20220831,20220930,20221031,20221130
티커,,,,,,,,,,,,,,,,,,,,,
005930,151328350.0,151328350.0,151328428.0,151389841.0,151877140.0,151931266.0,152108644.0,152178937.0,152717458.0,152717458.0,...,5.969783e+09,5.969783e+09,5.969783e+09,5.969783e+09,5.969783e+09,5.969783e+09,5.969783e+09,5.969783e+09,5.969783e+09,5.969783e+09
017670,89152670.0,89152670.0,89152670.0,89152670.0,89152670.0,89152670.0,89152670.0,89152670.0,89152670.0,89152670.0,...,2.188331e+08,2.188331e+08,2.188331e+08,2.188331e+08,2.188331e+08,2.188331e+08,2.188331e+08,2.188331e+08,2.188331e+08,2.188331e+08
030200,312199659.0,312199659.0,312199659.0,312199659.0,312199659.0,312199659.0,312199659.0,312199659.0,312199659.0,312199659.0,...,2.611118e+08,2.611118e+08,2.611118e+08,2.611118e+08,2.611118e+08,2.611118e+08,2.611118e+08,2.611118e+08,2.611118e+08,2.611118e+08
015760,640100876.0,640100876.0,640100876.0,640100876.0,640100876.0,640100876.0,640100876.0,640100876.0,640100876.0,640100876.0,...,6.419641e+08,6.419641e+08,6.419641e+08,6.419641e+08,6.419641e+08,6.419641e+08,6.419641e+08,6.419641e+08,6.419641e+08,6.419641e+08
060000,299697462.0,299697462.0,299697462.0,299697462.0,299697462.0,317677416.0,317677416.0,317677416.0,317677416.0,317677416.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.275190e+07
440200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.125000e+06
438220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.640000e+06


In [8]:
# 월별 누적 거래량 / 총 주식 발생 수 = 월별 Turn Over 
print("-----------------------------------------------월별 Turn Over-----------------------------------------------")
df_TO = df_q/df_num
df_TO

-----------------------------------------------월별 Turn Over-----------------------------------------------


,20011130,20011228,20020131,20020228,20020329,20020430,20020531,20020628,20020731,20020830,...,20220228,20220331,20220429,20220531,20220630,20220729,20220831,20220930,20221031,20221130
티커,,,,,,,,,,,,,,,,,,,,,
000010,0.412139,0.309391,0.426496,0.152995,0.219522,0.247889,0.171594,0.067728,0.074258,0.067863,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000020,1.155122,0.324466,0.723722,0.570271,0.657569,0.245165,0.132148,0.063117,0.114556,0.116420,...,0.213284,0.297665,0.075132,0.084694,0.120421,0.118533,0.072077,0.062754,0.042339,0.054394
000030,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000040,0.340008,0.125485,0.257276,0.134255,0.264620,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.029371,0.454839,0.052930,0.041343,0.058397,0.577881,0.070879,0.681409,0.482088,0.130862
000050,0.014062,0.067676,0.038468,0.032297,0.036630,0.089433,0.182807,0.064857,0.031122,0.034340,...,0.007088,0.009056,0.018826,0.019113,0.015863,0.003855,0.003737,0.004131,0.002756,0.004395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.059479,0.010103,0.004929,NaN,NaN,NaN,NaN,NaN,NaN,NaN
950190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.082107,0.093997,0.071469,0.039733,0.095534,0.059817,0.508202,0.067621,0.038164,0.036501
950200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.023567,0.019338,0.020097,0.017844,0.139068,0.142303,0.034907,1.000413,1.216384,0.134048


In [9]:
# 유(무)상 증(감)자와 액면분할을 고려한 수정 종가 구하기 : 5168일(252개월) 코스피 전종목에 대한 수정 종가
# ex) 삼성전자 같은 경우 50대 1 액면 분할을 하면서 주가가 1/50이 되었기 때문에 단순 종가를 사용하면 이를 주가하락으로 인식함
adj_close = pd.DataFrame()
all_stock = list(df_q.index.unique())
for i in all_stock:
    time.sleep(1) ; print(i, end = " ")
    temp = pd.DataFrame(stock.get_market_ohlcv("20011130", "20221130", i, adjusted=True)['종가'])
    temp.rename(columns = {'종가': i},inplace = True) ; temp = temp.T
    adj_close = pd.concat([adj_close,temp], axis = 0)

D = dict()
change = adj_close.columns.tolist()
for i in range(len(change)):
    D[change[i]] = datetime.datetime.strftime(change[i],"%Y%m%d")
    
adj_close.rename(columns = D, inplace = True)
print("-----------------------------------------------조정 종가-----------------------------------------------")
adj_close

-----------------------------------------------조정 종가-----------------------------------------------


,20011130,20011203,20011204,20011205,20011206,20011207,20011210,20011211,20011212,20011213,...,20221117,20221118,20221121,20221122,20221123,20221124,20221125,20221128,20221129,20221130
005930,4360.0,4490.0,4600.0,5290.0,5500.0,5670.0,5200.0,5170.0,5400.0,5300.0,...,61400.0,61800.0,61400.0,60600.0,61000.0,61400.0,61000.0,60100.0,60600.0,62200.0
017670,45550.0,44514.0,44859.0,49000.0,46412.0,46930.0,43565.0,44169.0,44946.0,43824.0,...,49850.0,49550.0,49750.0,49750.0,50000.0,50000.0,50000.0,49900.0,49800.0,50000.0
030200,53100.0,51900.0,51000.0,53000.0,51500.0,53300.0,50500.0,51400.0,51600.0,50900.0,...,36300.0,36600.0,36550.0,36850.0,37150.0,36650.0,37100.0,37150.0,37150.0,37400.0
015760,23000.0,22750.0,22500.0,23550.0,22750.0,23100.0,22200.0,22350.0,22700.0,22350.0,...,19550.0,19700.0,19500.0,19350.0,19900.0,19850.0,20150.0,20000.0,20250.0,20700.0
060000,46267.0,46550.0,46031.0,46928.0,45323.0,44945.0,43056.0,43765.0,44378.0,45039.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4600.0
440200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1980.0,1975.0,1980.0,1975.0,1975.0,1975.0,1975.0,1980.0,1975.0,1975.0
438220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0,2000.0,2000.0
440790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2000.0,2000.0,2000.0,1995.0,2000.0,1995.0,2000.0,1995.0,1990.0,1985.0


In [15]:
#월말 조정 종가
print("-----------------------------------------------월말 조정 종가-----------------------------------------------")
df_p = adj_close[days] 
df_p

-----------------------------------------------월말 조정 종가-----------------------------------------------


,20011130,20011228,20020131,20020228,20020329,20020430,20020531,20020628,20020731,20020830,...,20220228,20220331,20220429,20220531,20220630,20220729,20220831,20220930,20221031,20221130
005930,4360.0,5580.0,6010.0,6860.0,7360.0,7640.0,6840.0,6580.0,6650.0,6620.0,...,72100.0,69600.0,67400.0,67400.0,57000.0,61400.0,59700.0,53100.0,59400.0,62200.0
017670,45550.0,46240.0,44169.0,44859.0,50036.0,43393.0,46671.0,46499.0,39856.0,40805.0,...,54600.0,56900.0,57000.0,57100.0,52000.0,53600.0,52100.0,50800.0,50100.0,50000.0
030200,53100.0,49950.0,46950.0,58300.0,62300.0,58200.0,54200.0,48300.0,43850.0,54700.0,...,32000.0,35700.0,35800.0,37900.0,36450.0,37750.0,37200.0,36200.0,36600.0,37400.0
015760,23000.0,21700.0,21800.0,21950.0,24750.0,24500.0,25200.0,22000.0,21400.0,21050.0,...,23200.0,22650.0,23100.0,23350.0,22550.0,22350.0,20950.0,20100.0,16750.0,20700.0
060000,46267.0,49800.0,57700.0,57900.0,55500.0,58900.0,62700.0,58400.0,55000.0,56200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4600.0
440200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1975.0
438220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0
440790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985.0
